In [1]:
# https://twitter.com/p_surridge/status/925046405617594368

# antiIntellectual
# I’d rather put my trust in the wisdom of ordinary people than the opinions of experts
# W7W8W9W10W11

# education

# Any breakdown of these people by party vote or media consumption.
# It says alot by itself, but a semi unreality to it an actual practice!

# media source variables?

# profile_newspaper_readership_201 W10W13


# euSources W7W8

# infoSources W4W5W6W7W8W12W13

# tvWatch/radioListen/paperRead/internetRead W4W5W8

# twitterUse/twitterInfo W4W5W12

# fbUse/fbInfo           W4W5W12



# resourceAccess




In [2]:
import numpy as np
import pandas as pd
np.random.seed(0)

import matplotlib.pyplot as plt
import gc
import re

from sklearn import datasets
from sklearn.decomposition import PCA, IncrementalPCA, NMF, TruncatedSVD, FastICA, FactorAnalysis, SparsePCA
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from itertools import cycle
from IPython.display import display
import pickle, os

import seaborn as sns

# from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE
from gaussian_kde import gaussian_kde
from utility import display_components,display_pca_data


import time
import warnings

import numpy as np
import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

%matplotlib inline

In [3]:
from sklearn import datasets, cluster
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn import cluster

In [4]:
# you should clone this git to this subdirectory (in some directory - I call it BES_analysis - doesn't matter though)

if os.getcwd().split(os.sep)[-1] != 'BES_analysis_code':
    raise Exception("Stop! You're in the wrong directory - should be in 'BES_analysis_code'")

BES_code_folder   = "../BES_analysis_code/" # we should be here!
BES_small_data_files = BES_code_folder + "small data files" + os.sep
if not os.path.exists( BES_small_data_files ):
    os.makedirs( BES_small_data_files )

# we should create these if they don't already exist
BES_data_folder   = "../BES_analysis_data/"
if not os.path.exists( BES_data_folder ):
    os.makedirs( BES_data_folder )

BES_output_folder = "../BES_analysis_output/"
if not os.path.exists( BES_output_folder ):
    os.makedirs( BES_output_folder )

In [5]:
wave = "W8_comb"


In [6]:
import urllib.request
import zipfile
BES_file_manifest = pd.read_csv(BES_small_data_files+"BES_file_manifest.csv")

In [8]:

wave_list = BES_file_manifest.Name.values
# wave_list = ["W13_only"]

for wave in wave_list:


    manifest = BES_file_manifest[ BES_file_manifest["Name"] == wave ]

    filename = manifest["Stata_Filename"].values[0]
    # final filename! may need to be unzipped!

    url      = manifest["Stata_Url"].values[0]

    Treatment = wave
    
    data_subfolder = BES_data_folder+Treatment+os.sep
    
    # make data subfolder if it doesn't already exist
    
    if not os.path.exists(data_subfolder):
        os.makedirs(data_subfolder)

#     try:
#         BES_Panel = pd.read_stata(BES_data_folder+filename)

#     except FileNotFoundError:
    print("checking for "+wave+ " and file "+filename)
    
    

    # stata file does not exist, download!
    if not os.path.exists( data_subfolder + filename ):      


        opener=urllib.request.build_opener()
        opener.addheaders=[('User-Agent',
                            'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
        urllib.request.install_opener(opener)

        url_filename = url.split("/")[-1]
        print("downloading "+str(url_filename))        
        urllib.request.urlretrieve(url,
                                   data_subfolder + url_filename)

        # unzip?
        file_extension = url_filename.split(".")[-1]    
        if (file_extension == "zip"):
            print( "unzipping "+str( url_filename ) )
            zip_ref = zipfile.ZipFile( data_subfolder + url_filename , 'r')
            zip_ref.extractall( data_subfolder )
            zip_ref.close()
            print( "unzipped to "+str(filename) )
            
            # remove zip file!
            os.remove(data_subfolder + url_filename)
            print("deleted "+str(url_filename))

        elif (file_extension == "7z"):
            # sorry - no clean way to unzip this in python!
            raise Exception("I'm afraid you'll have extract this file manually: "+str(url_filename))
    else:
        print("It's there!")
#         BES_Panel = pd.read_stata( BES_data_folder+filename)

    # download the questionnaire pdf
    url_ques_filename = manifest[ "Ques_Url" ].values[0]
    ques_filename = url_ques_filename.split('/')[-1]
    if not os.path.exists( data_subfolder + ques_filename ):
        print( "downloading " + url_ques_filename )

        opener=urllib.request.build_opener()
        opener.addheaders=[('User-Agent',
                            'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
        urllib.request.install_opener(opener)

        urllib.request.urlretrieve(url_ques_filename,
                                   data_subfolder + ques_filename)        



checking for W13_only and file BES2017_W13_v1.0.dta
It's there!
checking for W13_comb and file BES2017_W13_Panel_v1.0.dta
It's there!
checking for W13_comb_strings and file BES2017_W13Strings_v1.0.dta
It's there!
checking for W12_only and file BES2015_W12_v1.0.dta
It's there!
checking for W11_only and file BES2015_W11_v1.0.dta
It's there!
checking for W10_comb and file BES2015_W10_Panel_v0.3.dta
It's there!
checking for W10_only and file BES2015_W10_v0.3.dta
It's there!
checking for W9_comb and file BES2015_W9_Panel_v1.2.dta
It's there!
checking for W9_only and file BES2015_W9_v1.2.dta
It's there!
checking for W8_only and file BES2015_W8_v1.6.dta
It's there!
checking for W8_comb and file BES2015_W8_Panel_v1.4.dta
It's there!
checking for W7_only and file BES2015_W7_v1.3.dta
It's there!
checking for W7_comb and file BES2015_W7_Panel_v3.0.dta
It's there!
checking for W6_only and file BES2015_W6_v3.6.dta
It's there!
checking for W6_comb and file BES2015_W6_Panel_v1.2.dta
It's there!
check

In [20]:
data_subfolder + filename

'../BES_analysis_data/W10_comb\\BES2015_W10_Panel_v0.3_stata.dta'

In [21]:
os.path.exists( data_subfolder + filename )

False

array(['W13_only', 'W13_comb', 'W13_comb_strings', 'W12_only', 'W11_only',
       'W10_comb', 'W10_only', 'W9_comb', 'W9_only', 'W8_only', 'W8_comb',
       'W7_only', 'W7_comb', 'W6_only', 'W6_comb', 'W5_comb', 'W5_only',
       'W4_only', 'W4_comb', 'W3_only', 'W3_comb', 'W2_only', 'W1_only'], dtype=object)

In [ ]:
BES_Panel["edlevel"].value_counts()

In [ ]:
BES_Panel["edlevel"].value_counts().sum()

In [ ]:
BES_Panel.shape

In [ ]:
BES_Panel["education"].value_counts()

In [ ]:
import seaborn as sns

In [ ]:
BES_Panel["antiIntellectual"].value_counts()[ BES_Panel["antiIntellectual"].cat.categories]

In [ ]:
BES_Panel["antiIntellectual"].cat.categories

In [ ]:
# I’d rather put my trust in the wisdom of ordinary people than the opinions of experts

sns.countplot(hue="antiIntellectual", x="edlevel", data=BES_Panel, palette="Greens_d");

In [ ]:
pd.crosstab(BES_Panel["antiIntellectual"],BES_Panel["edlevel"])

In [ ]:
N = 1000
mark = np.random.choice([True,False], N)
periods = np.random.choice(['BASELINE','WEEK 12', 'WEEK 24', 'WEEK 4'], N)

df = pd.DataFrame({'mark':mark,'period':periods})
ct = pd.crosstab(df.period, df.mark)

#  mark      False  True 
#  period                
#  BASELINE    118    111
#  WEEK 12     117    149
#  WEEK 24     117    130
#  WEEK 4      127    131

# now stack and reset
stacked = ct.stack().reset_index().rename(columns={0:'value'})

In [ ]:
var1 = "edlevel"
var2 = "antiIntellectual"

ct = pd.crosstab(BES_Panel[var1],BES_Panel[var2],normalize='index')*100
pct = ct.stack().reset_index().rename(columns={0:'percentage'})

fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(11.7, 8.27)

sns.barplot(hue=var2, x=var1, y='percentage', data=pct, palette="Greens_d",ax=ax);
ax.set_title("I’d rather put my trust in the wisdom of ordinary people than the opinions of experts")

fig.savefig( BES_data_folder+Treatment+"ed_by_anti_int_coarse.png", bbox_inches='tight' )


In [ ]:
var1 = "education"
var2 = "antiIntellectual"


ct = pd.crosstab(BES_Panel[var1],BES_Panel[var2],normalize='index')*100
pct = ct.stack().reset_index().rename(columns={0:'percentage'})

fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(11.7, 8.27)

sns.barplot(hue=var2, x=var1, y='percentage', data=pct, palette="Greens_d",ax=ax);
ax.set_title("I’d rather put my trust in the wisdom of ordinary people than the opinions of experts")
ax.set_xticklabels( BES_Panel[var1].cat.categories, rotation=90 ) ;

fig.savefig( BES_data_folder+Treatment+os.sep+"ed_by_anti_int_fine.png", bbox_inches='tight' )

In [ ]:
BES_Panel[var2].cat.remove_categories("Don't know")


not_dks = (BES_Panel[var2] != "Don't know")
BES_Panel[var2][not_dks].cat.codes.value_counts()

In [ ]:
var1 = "education"
var2 = "antiIntellectual"

fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(11.7, 8.27)


# get list of x-axis values ordered by mean
sorted_index = BES_Panel[var2][not_dks].cat.codes.groupby(BES_Panel[var1][not_dks]).mean().sort_values(ascending=True).index

sns.barplot(x=BES_Panel[var1][not_dks], y=BES_Panel[var2][not_dks].cat.codes, order = sorted_index)
ax.set_title("I’d rather put my trust in the wisdom of ordinary people than the opinions of experts")



ax.set_xticklabels( sorted_index, rotation=90 ) ;

fig.savefig( BES_data_folder+Treatment+os.sep+"ed_by_anti_int_coarse.png", bbox_inches='tight' )

In [ ]:
var1 = "generalElectionVote"
var2 = "antiIntellectual"

fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(11.7, 8.27)


# get list of x-axis values ordered by mean
sorted_index = BES_Panel[var2][not_dks].cat.codes.groupby(BES_Panel[var1][not_dks]).mean().sort_values(ascending=True).index

sns.barplot(x=BES_Panel[var1][not_dks], y=BES_Panel[var2][not_dks].cat.codes, order = sorted_index)
ax.set_title("I’d rather put my trust in the wisdom of ordinary people than the opinions of experts")



ax.set_xticklabels( sorted_index, rotation=90 ) ;

fig.savefig( BES_data_folder+Treatment+os.sep+"vote_by_anti_int_coarse.png", bbox_inches='tight' )

In [ ]:
var1 = "generalElectionVote"
var2 = "antiIntellectual"

ct = pd.crosstab(BES_Panel[var1],BES_Panel[var2],normalize='index')*100
pct = ct.stack().reset_index().rename(columns={0:'percentage'})

fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(11.7, 8.27)

sns.barplot(hue=var2, x=var1, y='percentage', data=pct, palette="Greens_d",ax=ax, order=sorted_index);
ax.set_title("I’d rather put my trust in the wisdom of ordinary people than the opinions of experts")
ax.set_xticklabels( sorted_index, rotation=90 ) ;

fig.savefig( BES_data_folder+Treatment+os.sep+"vote_by_anti_int_fine.png", bbox_inches='tight' )


In [ ]:
# wave 9 "profile_newspaper_readership_201"; wave 11 "profile_newspaper"
var1 = "profile_newspaper"
var2 = "antiIntellectual"

fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(11.7, 8.27)


# get list of x-axis values ordered by mean
sorted_index = BES_Panel[var2][not_dks].cat.codes.groupby(BES_Panel[var1][not_dks]).mean().sort_values(ascending=True).index

sns.barplot(x=BES_Panel[var1][not_dks], y=BES_Panel[var2][not_dks].cat.codes, order = sorted_index)
ax.set_title("I’d rather put my trust in the wisdom of ordinary people than the opinions of experts")



ax.set_xticklabels( sorted_index, rotation=90 ) ;

fig.savefig( BES_data_folder+Treatment+os.sep+"newspaper_by_anti_int_coarse.png", bbox_inches='tight' )

In [ ]:

ct = pd.crosstab(BES_Panel[var1],BES_Panel[var2],normalize='index')*100
pct = ct.stack().reset_index().rename(columns={0:'percentage'})

fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(11.7, 8.27)

sns.barplot(hue=var2, x=var1, y='percentage', data=pct, palette="Greens_d",ax=ax, order=sorted_index);
ax.set_title("I’d rather put my trust in the wisdom of ordinary people than the opinions of experts")
ax.set_xticklabels( sorted_index, rotation=90 ) ;

fig.savefig( BES_data_folder+Treatment+os.sep+"newspaper_by_anti_int_fine.png", bbox_inches='tight' )


In [ ]:
# [x for x in BES_Panel.columns if "profile_newspaper" in x]